In [1]:
import requests
import json
import time
import pandas as pd
from sqlalchemy import create_engine
import hashlib

### Get data

In [2]:
with open('all_detailed_players.json', 'r') as f:
    players = json.load(f)

In [3]:
with open('all_detailed_players_2.json', 'r') as f:
    players_2 = json.load(f)

In [7]:
combined_data = players + players_2

In [13]:
all_samples = []
for player_data in combined_data:
    answer = player_data['answer']
    player_info = answer['data']
    
    historical_matches = answer.get('points', [])
    if len(historical_matches) < 4:
        continue
    
    prices = [p['price'] for p in answer['prices']]
    max_price = max(prices)
    min_price = min(prices)
    for i in range(len(historical_matches) - 3):
        match_window = historical_matches[i:i+4]
        feature_matches = match_window[:3]
        target_match = match_window[3]
        
        feature_points = [m['points'] for m in feature_matches]
        last_3_average = sum(feature_points) / 3
        
        home_average = player_info['average']['homeAverage']
        away_average = player_info['average']['awayAverage']
        
        is_home_target = target_match['isHomeTeam']
        target_points = target_match['points']
        round_number = target_match['round']
        
        sample_dict = {
            'player_id': player_info['id'],
            'name': player_info['name'],
            'team': player_info['team'],
            'role': player_info['role'],
            'round': round_number,
            'home_average': home_average,
            'away_average': away_average,
            'overall_average': player_info['average']['average'],
            'last_3_average': last_3_average,
            'current_price': player_info['value'],
            'max_price': max_price,
            'min_price': min_price,
            'is_home_target': is_home_target,
            'match_minus_1': feature_matches[2]['points'],
            'match_minus_2': feature_matches[1]['points'],
            'match_minus_3': feature_matches[0]['points'],
            'target_points': target_points}
        
        all_samples.append(sample_dict)

df = pd.DataFrame(all_samples)

In [14]:
def generate_unique_id(row):
    unique_string = f"{row['player_id']}_{row['round']}"
    return hashlib.sha256(unique_string.encode()).hexdigest()

# Apply function to create new column
df['unique_id'] = df.apply(generate_unique_id, axis=1)

In [15]:
df.shape

(442, 18)

In [16]:
df_unique=df.drop_duplicates(subset=['unique_id'])

In [19]:
df.head(2)

,player_id,name,team,role,round,home_average,away_average,overall_average,last_3_average,current_price,max_price,min_price,is_home_target,match_minus_1,match_minus_2,match_minus_3,target_points,unique_id
0,504e58bb4d8bec9a67000187,Koke,Atlético de Madrid,centrocampista,4,5.833333,4.333333,5.083333,0.666667,21049633,21084293,20280310,True,2,0,0,11,d6162444b68251e9b3cf4ea125dd79267b06d698f05ad7...
1,504e58bb4d8bec9a67000187,Koke,Atlético de Madrid,centrocampista,5,5.833333,4.333333,5.083333,4.333333,21049633,21084293,20280310,False,11,2,0,7,92c1e291029079193c43855f54cce82a0e7b2147e6042d...


### Upload to Postgre

In [53]:
player_features=pd.read_csv('data/player_features.csv')

In [54]:
player_features.shape

(60, 23)

In [55]:
username = 'rodrigo'
# password = 'your_password'   
host = 'localhost'           
port = '5432'               
database = 'futmondo'

In [57]:
engine = create_engine(f'postgresql+psycopg2://{username}@{host}:{port}/{database}')

In [58]:
player_features.to_sql('players_features', engine, if_exists='replace', index=False)

60

In [59]:
query = "SELECT * FROM players_features"
df_from_db = pd.read_sql(query, engine)
df_from_db.shape

(60, 23)

In [61]:
df_from_db.head(2)

,player_id,name,team,role,total_points,matches_played,average,home_average,away_average,last_5_average,...,is_home_next,opponent_next,team_win_prob,draw_prob,opponent_win_prob,match_minus_1,match_minus_2,match_minus_3,match_minus_4,match_minus_5
0,504e58bb4d8bec9a67000187,Koke,Atlético de Madrid,centrocampista,61,12,5.083333,5.833333,4.333333,4.4,...,True,R. Oviedo,79.894209,13.554466,6.551325,2,5,5,8,2
1,55a975025bcb491d02476ab2,Álvaro Lemos,R. Oviedo,defensa,0,0,0.000000,0.000000,0.000000,0.0,...,False,Atlético de Madrid,6.551325,13.554466,79.894209,0,0,0,0,0
